# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,miandrivazo,-19.5167,45.4667,23.11,83,92,0.61,MG,1683325255
1,1,qarqaraly,49.4058,75.4742,7.86,45,6,1.48,KZ,1683325256
2,2,faya-largeau,17.9257,19.1043,32.09,11,0,3.74,TD,1683325256
3,3,thompson,55.7435,-97.8558,16.09,33,20,3.09,CA,1683325256
4,4,weno,7.4515,151.8468,28.97,79,100,1.54,FM,1683325256


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
# Configure the map plot
humidity_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    alpha = 0.5)

# Display the map
humidity_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df[((city_data_df["Humidity"] > 40) & 
                              (city_data_df["Humidity"] < 60)) & 
                              ((city_data_df["Max Temp"] <= 23) & 
                               (city_data_df["Max Temp"] >= 15)) & 
                               (city_data_df["Cloudiness"] >= 40)]

# Drop any rows with null values
ideal_city_df = ideal_city_df.dropna()

# Display sample data
ideal_city_df.head(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
49,49,albany,42.6001,-73.9662,17.47,50,96,1.79,US,1683325270
100,100,dursunlu,36.1628,36.1597,20.70,54,99,2.39,TR,1683325282
198,198,monterey,36.2505,-121.2516,15.50,57,93,3.48,US,1683325231
323,323,siwa oasis,29.2041,25.5195,21.33,45,41,6.39,EG,1683325336
324,324,kaiapoi,-43.3783,172.6401,19.15,57,100,3.60,NZ,1683325336
338,338,sarmiento,-45.5882,-69.0700,15.41,55,88,4.92,AR,1683325340
353,353,nueve de julio,-35.4444,-60.8831,18.31,49,99,0.89,AR,1683325344
364,364,laguna,38.4210,-121.4238,19.41,48,75,4.12,US,1683325347
366,366,palmer,42.1584,-72.3287,16.75,46,40,4.12,US,1683325348
412,412,conway,35.0887,-92.4421,22.89,47,100,3.09,US,1683325131


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df[["City", "Country", "Lat", "Lng", "Humidity"]]
pd.options.mode.chained_assignment = None

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(20)

,City,Country,Lat,Lng,Humidity,Hotel Name
49,albany,US,42.6001,-73.9662,50,
100,dursunlu,TR,36.1628,36.1597,54,
198,monterey,US,36.2505,-121.2516,57,
323,siwa oasis,EG,29.2041,25.5195,45,
324,kaiapoi,NZ,-43.3783,172.6401,57,
338,sarmiento,AR,-45.5882,-69.0700,55,
353,nueve de julio,AR,-35.4444,-60.8831,49,
364,laguna,US,38.4210,-121.4238,48,
366,palmer,US,42.1584,-72.3287,46,
412,conway,US,35.0887,-92.4421,47,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"

params = {
    "categories": categories,
    "apiKey": geoapify_key
    }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["datasource"]["raw"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
albany - nearest hotel: No hotel found
dursunlu - nearest hotel: Voyage Elegante Antakya
monterey - nearest hotel: No hotel found
siwa oasis - nearest hotel: فندق الكيلانى
kaiapoi - nearest hotel: No hotel found
sarmiento - nearest hotel: Ismar
nueve de julio - nearest hotel: Hotel Madrid
laguna - nearest hotel: Holiday Inn Express & Suites
palmer - nearest hotel: No hotel found
conway - nearest hotel: Fairfield Inn & Suites by Marriott Conway
waddan - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
yauya - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
49,albany,US,42.6001,-73.9662,50,No hotel found
100,dursunlu,TR,36.1628,36.1597,54,Voyage Elegante Antakya
198,monterey,US,36.2505,-121.2516,57,No hotel found
323,siwa oasis,EG,29.2041,25.5195,45,فندق الكيلانى
324,kaiapoi,NZ,-43.3783,172.6401,57,No hotel found
338,sarmiento,AR,-45.5882,-69.0700,55,Ismar
353,nueve de julio,AR,-35.4444,-60.8831,49,Hotel Madrid
364,laguna,US,38.4210,-121.4238,48,Holiday Inn Express & Suites
366,palmer,US,42.1584,-72.3287,46,No hotel found
412,conway,US,35.0887,-92.4421,47,Fairfield Inn & Suites by Marriott Conway


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
new_hotel_df = hotel_df[(hotel_df["Hotel Name"] != "No hotel found")]
new_hotel_df.head(10)

,City,Country,Lat,Lng,Humidity,Hotel Name
100,dursunlu,TR,36.1628,36.1597,54,Voyage Elegante Antakya
323,siwa oasis,EG,29.2041,25.5195,45,فندق الكيلانى
338,sarmiento,AR,-45.5882,-69.0700,55,Ismar
353,nueve de julio,AR,-35.4444,-60.8831,49,Hotel Madrid
364,laguna,US,38.4210,-121.4238,48,Holiday Inn Express & Suites
412,conway,US,35.0887,-92.4421,47,Fairfield Inn & Suites by Marriott Conway


In [16]:
# Configure the map plot
hotel_plot = new_hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    alpha = 0.5,
    hover_cols = ["Hotel Name", "Country"])

# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)